In [48]:
""" This script tries only using the TuLiP toolbox to solve a multirobot motion planning problem.
Specifically, a field of cells and a number of robots are specified along with the system dynamics,
and constraints are written in monolithic LTL syntax.
The difference between this script and TuLiP_multirobot_solver.py
"""

# Import the packages that we need
from __future__ import print_function
import numpy as np
from tulip import spec
from tulip.transys import machines
from tulip import synth
import gr1_fragment
import State_encoding
import LTL_encoding

In [49]:
# Workspace parameters
num_rows = 3
num_cols = 3
num_robots = 2
delay = 2
robot_init_pos = np.array([[0, 1], [1, 0]])
# The environment variables
env_vars = {}
env_init = set()
env_prog = set()
env_safe = set()

In [50]:
# Define sys_vars
sys_vars = {}
for r in range(1, num_robots + 1):
    sys_vars[('x' + str(r))] = (0, num_cols)
    sys_vars[('y' + str(r))] = (0, num_rows)
for r in range(1, num_robots + 1):
    sys_vars['stop' + str(r)] = 'boolean'
    sys_vars['left' + str(r)] = 'boolean'
    sys_vars['right' + str(r)] = 'boolean'
    sys_vars['up' + str(r)] = 'boolean'
    sys_vars['down' + str(r)] = 'boolean'
    sys_vars[('c' + str(r))] = (-1, delay + 1)
    sys_vars[('delay' + str(r))] = 'boolean'
print(sys_vars)

{'x1': (0, 3), 'y1': (0, 3), 'x2': (0, 3), 'y2': (0, 3), 'stop1': 'boolean', 'left1': 'boolean', 'right1': 'boolean', 'up1': 'boolean', 'down1': 'boolean', 'c1': (-1, 3), 'delay1': 'boolean', 'stop2': 'boolean', 'left2': 'boolean', 'right2': 'boolean', 'up2': 'boolean', 'down2': 'boolean', 'c2': (-1, 3), 'delay2': 'boolean'}


In [51]:
# Define initial command.
sys_init = set()
for r in range(1, num_robots + 1):
    sys_init |= {'x' + str(r) + '=' + str(robot_init_pos[r - 1][0])}
    sys_init |= {'y' + str(r) + '=' + str(robot_init_pos[r - 1][1])}
    sys_init |= {'c' + str(r) + '=0'}
print(sys_init)

{'delay1 && X(!delay1) && !delay2 && X(delay2)', 'y1=1', 'c2=0', 'x1=0', 'y2=0', 'x2=1', 'c1=0'}


In [52]:
sys_safe = set()
sys_prog = set()

In [53]:
# Now no need to encode that each robot should only appear in one position
# Encode the transition relations
sys_trans_x = set()
sys_trans_y = set()
for r in range(1, num_robots + 1):
    for col in range(num_cols):
        if col != num_cols - 1:
            sys_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && right' + str(r) + ' ) -> ( ( X(x' + str(r) + '=' +
                            str(col+1) + ') && !delay' + str(r) + ' ) || ( X(x' + str(r) + '=' + str(col) +
                            ' && right' + str(r) + ') && delay' + str(r) + ' ) )'}
        if col != 0:
            sys_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && left' + str(r) + ' ) -> ( ( X(x' + str(r) + '=' +
                            str(col - 1) + ') && !delay' + str(r) + ' ) || ( X(x' + str(r) + '=' + str(col) +
                            ' && left' + str(r) + ') && delay' + str(r) + ' ) )'}
        sys_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && up' + str(r) + ' ) -> X( x' + str(r) + '=' + str(col) +
                        ' )'}
        sys_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && down' + str(r) + ' ) -> X( x' + str(r) + '=' + str(col) +
                        ' )'}
        sys_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && stop' + str(r) + ' ) -> ( X( x' + str(r) + '=' + str(col) +
                        ' ) && ( !delay' + str(r) + ' ) )'}
    for row in range(num_rows):
        if row != num_rows - 1:
            sys_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && up' + str(r) + ' ) -> ( ( X(y' + str(r) + '=' +
                            str(row + 1) + ') && !delay' + str(r) + ' ) || ( X(y' + str(r) + '=' + str(row) +
                            ' && up' + str(r) + ') && delay' + str(r) + ' ) )'}
        if row != 0:
            sys_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && down' + str(r) + ' ) -> ( ( X(y' + str(r) + '=' +
                            str(row - 1) + ') && !delay' + str(r) + ' ) || ( X(y' + str(r) + '=' + str(row) +
                            ' && down' + str(r) + ') && delay' + str(r) + ' ) )'}
        sys_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && left' + str(r) + ' ) -> X( y' + str(r) + '=' + str(row) +
                        ' )'}
        sys_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && right' + str(r) + ' ) -> X( y' + str(r) + '=' + str(row) +
                        ' )'}
        sys_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && stop' + str(r) + ' ) -> ( X( y' + str(r) + '=' + str(row) +
                        ' ) && ( !delay' + str(r) + ' ) )'}
print(sys_trans_x)
print(sys_trans_y)
sys_safe |= sys_trans_x
sys_safe |= sys_trans_y

{'( x2=0 && down2 ) -> X( x2=0 )', '( x1=0 && stop1 ) -> ( X( x1=0 ) && ( !delay1 ) )', '( x2=2 && left2 ) -> ( ( X(x2=1) && !delay2 ) || ( X(x2=2 && left2) && delay2 ) )', '( x2=0 && up2 ) -> X( x2=0 )', '( x1=1 && up1 ) -> X( x1=1 )', '( x2=0 && right2 ) -> ( ( X(x2=1) && !delay2 ) || ( X(x2=0 && right2) && delay2 ) )', '( x1=1 && right1 ) -> ( ( X(x1=2) && !delay1 ) || ( X(x1=1 && right1) && delay1 ) )', '( x1=1 && left1 ) -> ( ( X(x1=0) && !delay1 ) || ( X(x1=1 && left1) && delay1 ) )', '( x1=1 && down1 ) -> X( x1=1 )', '( x2=2 && down2 ) -> X( x2=2 )', '( x2=2 && up2 ) -> X( x2=2 )', '( x1=2 && left1 ) -> ( ( X(x1=1) && !delay1 ) || ( X(x1=2 && left1) && delay1 ) )', '( x1=0 && right1 ) -> ( ( X(x1=1) && !delay1 ) || ( X(x1=0 && right1) && delay1 ) )', '( x2=1 && down2 ) -> X( x2=1 )', '( x2=2 && stop2 ) -> ( X( x2=2 ) && ( !delay2 ) )', '( x1=0 && up1 ) -> X( x1=0 )', '( x1=0 && down1 ) -> X( x1=0 )', '( x1=1 && stop1 ) -> ( X( x1=1 ) && ( !delay1 ) )', '( x2=1 && left2 ) -> ( ( 

In [54]:
# Relate the delay variables (delay#) to the clock variables (c#)
delay2clock_minus = '( '
delay2clock_plus = ''
for r in range(1, num_robots + 1):
    delay2clock_minus += '( delay' + str(r) + ' && c' + str(r) + '=' + str(delay) + ' ) || '
delay2clock_plus = '( !' + delay2clock_minus[:-3] + ') ) -> ( '
delay2clock_minus = delay2clock_minus[:-3] + ') -> ( '
for r in range(1, num_robots + 1):
    for c in range(delay + 1):
        delay2clock_minus += '( ( !delay' + str(r) + ' && c' + str(r) + '=' + str(c) + ' ) -> X(c' + str(r) + '=' + \
                             str(c - 1) + ') ) && '
        delay2clock_minus += '( ( delay' + str(r) + ' && c' + str(r) + '=' + str(c) + ' ) -> X(c' + str(r) + '=' + \
                             str(c) + ') ) && '
        delay2clock_plus += '( ( delay' + str(r) + ' && c' + str(r) + '=' + str(c) + ' ) -> X(c' + str(r) + '=' + \
                            str(c + 1) + ') ) && '
        delay2clock_plus += '( ( !delay' + str(r) + ' && c' + str(r) + '=' + str(c) + ' ) -> X(c' + str(r) + '=' + \
                            str(c) + ') ) && '
delay2clock_minus = delay2clock_minus[:-3] + ')'
delay2clock_plus = delay2clock_plus[:-3] + ')'
print(delay2clock_minus)
print(delay2clock_plus)
sys_safe |= {delay2clock_minus}
sys_safe |= {delay2clock_plus}

( ( delay1 && c1=2 ) || ( delay2 && c2=2 ) ) -> ( ( ( !delay1 && c1=0 ) -> X(c1=-1) ) && ( ( delay1 && c1=0 ) -> X(c1=0) ) && ( ( !delay1 && c1=1 ) -> X(c1=0) ) && ( ( delay1 && c1=1 ) -> X(c1=1) ) && ( ( !delay1 && c1=2 ) -> X(c1=1) ) && ( ( delay1 && c1=2 ) -> X(c1=2) ) && ( ( !delay2 && c2=0 ) -> X(c2=-1) ) && ( ( delay2 && c2=0 ) -> X(c2=0) ) && ( ( !delay2 && c2=1 ) -> X(c2=0) ) && ( ( delay2 && c2=1 ) -> X(c2=1) ) && ( ( !delay2 && c2=2 ) -> X(c2=1) ) && ( ( delay2 && c2=2 ) -> X(c2=2) ) )
( !( ( delay1 && c1=2 ) || ( delay2 && c2=2 ) ) ) -> ( ( ( delay1 && c1=0 ) -> X(c1=1) ) && ( ( !delay1 && c1=0 ) -> X(c1=0) ) && ( ( delay1 && c1=1 ) -> X(c1=2) ) && ( ( !delay1 && c1=1 ) -> X(c1=1) ) && ( ( delay1 && c1=2 ) -> X(c1=3) ) && ( ( !delay1 && c1=2 ) -> X(c1=2) ) && ( ( delay2 && c2=0 ) -> X(c2=1) ) && ( ( !delay2 && c2=0 ) -> X(c2=0) ) && ( ( delay2 && c2=1 ) -> X(c2=2) ) && ( ( !delay2 && c2=1 ) -> X(c2=1) ) && ( ( delay2 && c2=2 ) -> X(c2=3) ) && ( ( !delay2 && c2=2 ) -> X(c2=2)

In [55]:
# Constraint on the maximum delay
sys_max_delay = set()
for r in range(1, num_robots + 1):
    sys_max_delay |= {'!(c' + str(r) + '=' + '-1)'}
    sys_max_delay |= {'!(c' + str(r) + '=' + str(delay + 1) + ')'}
print(sys_max_delay)
sys_safe |= sys_max_delay

{'!(c2=-1)', '!(c2=3)', '!(c1=-1)', '!(c1=3)'}


In [56]:
# Avoid bad actions
sys_bad_act = set()
for r in range(1, num_robots + 1):
    sys_bad_act |= {'x' + str(r) + '=0 -> !left' + str(r)}
    sys_bad_act |= {'x' + str(r) + '=' + str(num_cols - 1) + ' -> !right' + str(r)}
    sys_bad_act |= {'y' + str(r) + '=0 -> !down' + str(r)}
    sys_bad_act |= {'y' + str(r) + '=' + str(num_rows - 1) + ' -> !up' + str(r)}
print(sys_bad_act)
sys_safe |= sys_bad_act

{'x2=2 -> !right2', 'x1=2 -> !right1', 'x2=0 -> !left2', 'x1=0 -> !left1', 'y1=2 -> !up1', 'y2=0 -> !down2', 'y2=2 -> !up2', 'y1=0 -> !down1'}


In [57]:
# Encode the constraint that each time exactly one action must be chosen
sys_good_act = set()
for r in range(1, num_robots + 1):
    sys_good_act |= {'( left' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( right' + str(r) + ' ) <-> ( ' + '!left' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( up' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!left' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( down' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!left' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( stop' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!left' + str(r) + ' )'}
sys_safe |= sys_good_act
print(sys_good_act)

{'( left1 ) <-> ( !right1 && !up1 && !down1 && !stop1 )', '( stop1 ) <-> ( !right1 && !up1 && !down1 && !left1 )', '( right2 ) <-> ( !left2 && !up2 && !down2 && !stop2 )', '( up2 ) <-> ( !right2 && !left2 && !down2 && !stop2 )', '( left2 ) <-> ( !right2 && !up2 && !down2 && !stop2 )', '( right1 ) <-> ( !left1 && !up1 && !down1 && !stop1 )', '( stop2 ) <-> ( !right2 && !up2 && !down2 && !left2 )', '( down2 ) <-> ( !right2 && !up2 && !left2 && !stop2 )', '( down1 ) <-> ( !right1 && !up1 && !left1 && !stop1 )', '( up1 ) <-> ( !right1 && !left1 && !down1 && !stop1 )'}


In [58]:
# User-defined system requirements
# Constraint 1: []<>([[0, 0], [0, 1], [0, 2]], 2)
# Constraint 2: []!([1, 1], 1)
# Constraint 3: ([]<>[2, 2], 2)

sys_prog |= {'(x1=0) && (x2=0)'}
sys_safe |= {'!(x1=1 && y1=1) && !(x2=1 && y2=1)'}
sys_prog |= {'x1=2 && y1=2'}
sys_prog |= {'x2=2 && y2=2'}

In [59]:
# Create a GR(1) specification
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init,
                    env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\E \A'  # Moore initial condition synthesized too
specs.moore = False
specs.plus_one = False

In [60]:
specs.check_syntax()

AssertionError: Syntax error: primed variables: {'delay2', 'delay1'} found in guaranteed initial condition: delay1 && X(!delay1) && !delay2 && X(delay2)

In [45]:
print(specs)

(delay1) && (y1=1) && (c2=0) && (x1=0) && (y2=0) && (x2=1) && (c1=0) && [](x2=0 -> !left2) && [](( y1=0 && right1 ) -> X( y1=0 )) && [](( x1=0 && stop1 ) -> ( X( x1=0 ) && ( !delay1 ) )) && [](( y2=0 && right2 ) -> X( y2=0 )) && [](( y2=2 && right2 ) -> X( y2=2 )) && [](( y2=1 && left2 ) -> X( y2=1 )) && [](( x2=0 && up2 ) -> X( x2=0 )) && [](y2=2 -> !up2) && [](( x2=0 && right2 ) -> ( ( X(x2=1) && !delay2 ) || ( X(x2=0 && right2) && delay2 ) )) && [](( y1=2 && right1 ) -> X( y1=2 )) && [](( y1=2 && left1 ) -> X( y1=2 )) && [](( !( ( delay1 && c1=2 ) || ( delay2 && c2=2 ) ) ) -> ( ( ( delay1 && c1=0 ) -> X(c1=1) ) && ( ( !delay1 && c1=0 ) -> X(c1=0) ) && ( ( delay1 && c1=1 ) -> X(c1=2) ) && ( ( !delay1 && c1=1 ) -> X(c1=1) ) && ( ( delay1 && c1=2 ) -> X(c1=3) ) && ( ( !delay1 && c1=2 ) -> X(c1=2) ) && ( ( delay2 && c2=0 ) -> X(c2=1) ) && ( ( !delay2 && c2=0 ) -> X(c2=0) ) && ( ( delay2 && c2=1 ) -> X(c2=2) ) && ( ( !delay2 && c2=1 ) -> X(c2=1) ) && ( ( delay2 && c2=2 ) -> X(c2=3) ) && 

In [46]:
print('Start synthesis')
ctrl = synth.synthesize(specs)
print('End synthesis')
assert ctrl is not None, 'unrealizable'

Start synthesis
removed 0 nodes from 26 total
End synthesis


In [47]:
machines.random_run(ctrl, N=10)

move from
	 state: Sinit
	 with input:{}
	 to state: 0
	 reacting by producing output: {'stop1': False, 'left1': False, 'right1': False, 'up1': False, 'down1': True, 'delay1': True, 'stop2': False, 'left2': False, 'right2': True, 'up2': False, 'down2': False, 'delay2': False, 'x1': 0, 'y1': 1, 'x2': 1, 'y2': 0, 'c1': 0, 'c2': 0}
move from
	 state: 0
	 with input:{}
	 to state: 1
	 reacting by producing output: {'stop1': False, 'left1': False, 'right1': False, 'up1': False, 'down1': True, 'delay1': False, 'stop2': False, 'left2': False, 'right2': False, 'up2': True, 'down2': False, 'delay2': False, 'x1': 0, 'y1': 1, 'x2': 2, 'y2': 0, 'c1': 1, 'c2': 0}
move from
	 state: 1
	 with input:{}
	 to state: 2
	 reacting by producing output: {'stop1': False, 'left1': False, 'right1': False, 'up1': True, 'down1': False, 'delay1': False, 'stop2': False, 'left2': False, 'right2': False, 'up2': False, 'down2': True, 'delay2': False, 'x1': 0, 'y1': 0, 'x2': 2, 'y2': 1, 'c1': 1, 'c2': 0}
move from
	 s

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 {'x1': [0, 0, 0, 0, 0, 1, 2, 2, 2, 2],
  'y1': [1, 1, 0, 1, 2, 2, 2, 1, 0, 1],
  'x2': [1, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  'y2': [0, 0, 1, 0, 1, 0, 1, 0, 1, 2],
  'stop1': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'left1': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'right1': [False,
   False,
   False,
   False,
   True,
   True,
   False,
   False,
   False,
   False],
  'up1': [False, False, True, True, False, False, False, False, True, False],
  'down1': [True, True, False, False, False, False, True, True, False, True],
  'c1': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'delay1': [True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'stop2': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'left2': [False,
   False,
   False,
   False,
   False,
   